In [1]:
import pandas as pd

# Read all parquet files in the 'data' directory into a single DataFrame
df = pd.read_parquet("data")

print(f"Loaded {len(df)} rows and {len(df.columns)} columns.")
print(df.head())

Loaded 655785 rows and 4 columns.
  carpark_id    carpark_name vendor  \
0      mitte  Parkhaus Mitte    MSR   
1      mitte  Parkhaus Mitte    MSR   
2      mitte  Parkhaus Mitte    MSR   
3      mitte  Parkhaus Mitte    MSR   
4      mitte  Parkhaus Mitte    MSR   

                                      current  
0  {'timestamp': 1728916440012, 'value': 218}  
1  {'timestamp': 1728916560015, 'value': 215}  
2  {'timestamp': 1728916680007, 'value': 209}  
3  {'timestamp': 1728916800007, 'value': 206}  
4  {'timestamp': 1728916920011, 'value': 200}  


In [2]:
current_df = pd.json_normalize(df["current"])
df = pd.concat([df.drop(columns=["current"]), current_df], axis=1)
df

,carpark_id,carpark_name,vendor,timestamp,value
0,mitte,Parkhaus Mitte,MSR,1728916440012,218
1,mitte,Parkhaus Mitte,MSR,1728916560015,215
2,mitte,Parkhaus Mitte,MSR,1728916680007,209
3,mitte,Parkhaus Mitte,MSR,1728916800007,206
4,mitte,Parkhaus Mitte,MSR,1728916920011,200
...,...,...,...,...,...
655780,mitte,Parkhaus Mitte,MSR,1760651760002,19
655781,mitte,Parkhaus Mitte,MSR,1760651820003,20
655782,mitte,Parkhaus Mitte,MSR,1760651880003,25
655783,mitte,Parkhaus Mitte,MSR,1760651940004,19


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655785 entries, 0 to 655784
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   carpark_id    655785 non-null  object
 1   carpark_name  655785 non-null  object
 2   vendor        655785 non-null  object
 3   timestamp     655785 non-null  int64 
 4   value         655785 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 25.0+ MB


In [4]:
import hopsworks
proj = hopsworks.login()
fs = proj.get_feature_store()

fg = fs.get_or_create_feature_group(name="parking", 
                                    version=1,
                                    primary_key=['carpark_id'], 
                                    event_time="timestamp",
                                    description="Parking occupancy data for Heilbronn"
                                   )
fg.insert(df)

2025-10-22 15:08:23,202 INFO: Python Engine initialized.

Logged in to project, explore it here https://snurran.devnet.hops.works/p/5243
Feature Group created successfully, explore it at 
https://snurran.devnet.hops.works/p/5243/fs/5191/fg/5159


Uploading Dataframe: 100.00% |██████████| Rows 655785/655785 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: parking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.devnet.hops.works/p/5243/jobs/named/parking_1_offline_fg_materialization/executions


(Job('parking_1_offline_fg_materialization', 'SPARK'), None)